In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,date

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', None)

In [12]:
# 2021-6-21:双状况计划监控模型——末日广点通
def source_predict_state_1_gdt(jsondata):
    '''
    :param jsondata: {"columns": ["channel_id","source_id","plan_name", "model_run_datetime","create_time","media_id",
    "game_id","platform","source_run_date_amount","create_role_num","create_role_cost","create_role_pay_num",
    "create_role_pay_cost","create_role_pay_sum","create_role_roi","create_role_pay_rate","create_role_pay_num_cum",
    "learning_type","learning_time_dt","learning_time_hr","deep_bid_type"]}
    :return:'label','budget'
    '''
    
    # 该模型只能接受计划状态plan_run_state=1且data_win=0 且media_id=16 且 model_run_datetime 等于当日的数据，即只接受当日广点通实时数据
    # label为1时关计划，为0时开计划，如果模型输出label结果与计划当前状况（开/关）相同，则不操作。例如，计划当前是关停状况，当模型输出为1时，不操作

    # 早上7：55分，将所有在运行的计划状态plan_run_state改为2。调用模型2进行判断，是否达标。达标则状态改为1，开计划，预算由模型1计算决定。不达标，则为关停状态，且plan_run_state为2
        
    
    

    data = pd.DataFrame(data=jsondata['data'], columns=jsondata['columns'])
    data = data[data['media_id']==16]
    # 数据预处理
    data = data[data['source_run_date_amount'] > 0]  # 删除消耗为0的数据

    # 对消耗300元以下的不作判断，对数据按平台、win进行分类处理
    data_not_ignore = data[data['source_run_date_amount'] >= 300]

    # 增加每次付费跑法的判断
#     data_pre_action = data_not_ignore[data_not_ignore['deep_bid_type'] == 'BID_PER_ACTION']
#     data_not_ignore = data_not_ignore[data_not_ignore['deep_bid_type'] != 'BID_PER_ACTION']
    
    data_win_0 = data_not_ignore[data_not_ignore['platform'] == 1]
    data_win_0_ios = data_not_ignore[data_not_ignore['platform'] == 2]

#     data_win_0_pre = data_pre_action[data_pre_action['platform'] == 1]
#     data_win_0_ios_pre = data_pre_action[data_pre_action['platform'] == 2]

    result_df = pd.DataFrame()
    
    # 1:关; 0:开； 2：保持原状
    # win=0预判
    if data_win_0.shape[0] != 0:
        data_win_0_1 = data_win_0[data_win_0['source_run_date_amount']<=7000]
        data_win_0_2 = data_win_0[(data_win_0['source_run_date_amount']>7000)&(data_win_0['source_run_date_amount']<=13000)]
        data_win_0_3 = data_win_0[(data_win_0['source_run_date_amount']>13000)&(data_win_0['source_run_date_amount']<=25000)]
        data_win_0_4 = data_win_0[(data_win_0['source_run_date_amount']>25000)&(data_win_0['source_run_date_amount']<=50000)]
        data_win_0_5 = data_win_0[data_win_0['source_run_date_amount']>50000]
        if data_win_0_1.shape[0] != 0:
            data_win_0_1['label'] = data_win_0_1.apply(
                lambda x: 1 if (((x.create_role_cost >= 250) & (x.source_run_date_amount <= 1000) & (x.create_role_pay_sum == 0)) |
                                ((x.create_role_cost >= 300) & (x.source_run_date_amount <= 1000) & (x.create_role_pay_sum != 0)) |
                                ((x.create_role_cost >= 140) & (x.source_run_date_amount > 1000) & (x.create_role_pay_sum == 0)) |
                                ((x.create_role_cost >= 180) & (x.source_run_date_amount > 1000) & (x.create_role_pay_sum != 0)) |
                                ((x.create_role_roi <= 0.018) & (x.source_run_date_amount >= 3500))) else (0
                            if (((x.create_role_cost <= 200) & (x.source_run_date_amount <= 1000) & (x.create_role_pay_sum == 0)) |
                                ((x.create_role_cost <= 250) & (x.source_run_date_amount <= 1000) & (x.create_role_pay_sum != 0)) |
                                ((x.create_role_cost <= 120) & (x.source_run_date_amount > 1000) & (x.create_role_pay_sum == 0)) |
                                ((x.create_role_cost <= 150) & (x.source_run_date_amount > 1000) & (x.create_role_pay_sum != 0)) |
                                (x.create_role_roi >= 0.02)) else 2), axis=1)   
            result_df = result_df.append(data_win_0_1)
        
        if data_win_0_2.shape[0] != 0:
            data_win_0_2['label'] = data_win_0_2.apply(lambda x:1 if (x.create_role_cost>= 250) else (1 if (x.create_role_roi <=0.025)
                            |(x.create_role_pay_cost>=8000) else (0 if (x.create_role_roi >=0.035) else 2)), axis=1)    
            result_df = result_df.append(data_win_0_2)
        
        if data_win_0_3.shape[0] != 0:
            data_win_0_3['label'] = data_win_0_3.apply(lambda x:1 if (x.create_role_cost>= 250) else (1 if (x.create_role_roi <=0.03)
                            |(x.create_role_pay_cost>=8000) else (0 if (x.create_role_roi >=0.04) else 2)), axis=1)    
            result_df = result_df.append(data_win_0_3)
        
        if data_win_0_4.shape[0] != 0:
            data_win_0_4['label'] = data_win_0_4.apply(lambda x:1 if (x.create_role_cost>= 250) else (1 if (x.create_role_roi <=0.035)
                            |(x.create_role_pay_cost>=7000) else (0 if (x.create_role_roi >=0.045) else 2)), axis=1)    
            result_df = result_df.append(data_win_0_4)
            
        if data_win_0_5.shape[0] != 0:
            data_win_0_5['label'] = data_win_0_5.apply(lambda x:1 if (x.create_role_cost>= 250) else (1 if (x.create_role_roi <=0.035)
                            |(x.create_role_pay_cost>=6500) else (0 if (x.create_role_roi >=0.045) else 2)), axis=1)    
            result_df = result_df.append(data_win_0_5)

    # ISO
    if data_win_0_ios.shape[0] != 0:
        data_win_0_ios_1 = data_win_0_ios[data_win_0_ios['source_run_date_amount']<=7000]
        data_win_0_ios_2 = data_win_0_ios[(data_win_0_ios['source_run_date_amount']>7000)&(data_win_0_ios['source_run_date_amount']<=13000)]
        data_win_0_ios_3 = data_win_0_ios[(data_win_0_ios['source_run_date_amount']>13000)&(data_win_0_ios['source_run_date_amount']<=25000)]
        data_win_0_ios_4 = data_win_0_ios[(data_win_0_ios['source_run_date_amount']>25000)&(data_win_0_ios['source_run_date_amount']<=50000)]
        data_win_0_ios_5 = data_win_0_ios[data_win_0_ios['source_run_date_amount']>50000]
        if data_win_0_ios_1.shape[0] != 0:
            data_win_0_ios_1['label'] = data_win_0_ios_1.apply(lambda x:1 if (x.create_role_cost>= 500)&(x.deep_bid_type== 'BID_PER_ACTION')| 
                (x.create_role_cost>= 350)&(x.deep_bid_type!= 'BID_PER_ACTION') else (1 if (x.create_role_roi <=0.02)
                            &(x.source_run_date_amount>=5000) else (0 if (x.create_role_roi >=0.03) else 2)), axis=1)    
            result_df = result_df.append(data_win_0_ios_1)
        
        if data_win_0_ios_2.shape[0] != 0:
            data_win_0_ios_2['label'] = data_win_0_ios_2.apply(lambda x:1 if (x.create_role_cost>= 350) else (1 if (x.create_role_roi <=0.025)
                            |(x.create_role_pay_cost>=8000) else (0 if (x.create_role_roi >=0.035) else 2)), axis=1)    
            result_df = result_df.append(data_win_0_ios_2)
        
        if data_win_0_ios_3.shape[0] != 0:
            data_win_0_ios_3['label'] = data_win_0_ios_3.apply(lambda x:1 if (x.create_role_cost>= 350) else (1 if (x.create_role_roi <=0.03)
                            |(x.create_role_pay_cost>=8000) else (0 if (x.create_role_roi >=0.04) else 2)), axis=1)    
            result_df = result_df.append(data_win_0_ios_3)
        
        if data_win_0_ios_4.shape[0] != 0:
            data_win_0_ios_4['label'] = data_win_0_ios_4.apply(lambda x:1 if (x.create_role_cost>= 350) else (1 if (x.create_role_roi <=0.035)
                            |(x.create_role_pay_cost>=7000) else (0 if (x.create_role_roi >=0.045) else 2)), axis=1)    
            result_df = result_df.append(data_win_0_ios_4)
            
        if data_win_0_ios_5.shape[0] != 0:
            data_win_0_ios_5['label'] = data_win_0_ios_5.apply(lambda x:1 if (x.create_role_cost>= 350) else (1 if (x.create_role_roi <=0.035)
                            |(x.create_role_pay_cost>=6500) else (0 if (x.create_role_roi >=0.045) else 2)), axis=1)    
            result_df = result_df.append(data_win_0_ios_5)
    
    result_df['label'] = result_df['label'].astype(int)
    return {"columns": result_df.columns.values.tolist(), "list": result_df.values.tolist()}

In [35]:
jsondata = {"columns": state_1_data.columns,
           "data":state_1_data.values}
columns_1 = source_predict_state_1_gdt(jsondata)['columns']
data_1 = source_predict_state_1_gdt(jsondata)['list']
df1 = pd.DataFrame(data_1,columns=columns_1)

In [36]:
df1

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_pay_rate,create_role_pay_num_cum,learning_type,learning_time_dt,learning_time_hr,deep_bid_type,label
0,20451,493367,303-幸存者-ROI自动02.18-5,2021-02-22 00:00:00,2021-02-18 00:00:00,16,1001379,1,2000,349,119,3,3200,0,0.0,0.01,2,2,NaN,NaN,BID_PER_ACTION,0


In [21]:
state_1_data

,channel_id,source_id,plan_name,model_run_datetime,create_time,media_id,game_id,platform,source_run_date_amount,create_role_num,create_role_cost,create_role_pay_num,create_role_pay_cost,create_role_pay_sum,create_role_roi,create_role_pay_rate,create_role_pay_num_cum,learning_type,learning_time_dt,learning_time_hr,deep_bid_type
0,20451,493367,303-幸存者-ROI自动02.18-5,2021-02-22 00:00:00,2021-02-18 00:00:00,16,1001379,1,6000,349,100,3,3200,200,0.019,0.01,2,2,NaN,NaN,BID_PER_ACTION


In [34]:
state_1_data = pd.DataFrame(data=[[
                20451,
                493367,
                "303-幸存者-ROI自动02.18-5",
                "2021-02-22 00:00:00",
                "2021-02-18 00:00:00",
                16,
                1001379,
                1,
                2000,
                349,
                119,
                3,
                3200,
                0,
                0.0,
                0.01,
                2,
                2,
                np.nan,
                np.nan,
                "BID_PER_ACTION"
            ]],
            columns = [
            "channel_id",
            "source_id",
            "plan_name",
            "model_run_datetime",
            "create_time",
            "media_id",
            "game_id",
            "platform",
            "source_run_date_amount",
            "create_role_num",
            "create_role_cost",
            "create_role_pay_num",
            "create_role_pay_cost",
            "create_role_pay_sum",
            "create_role_roi",
            "create_role_pay_rate",
            "create_role_pay_num_cum",
            "learning_type",
            "learning_time_dt",
            "learning_time_hr",
            "deep_bid_type"])